    INFO: sleutels voor bookmakers: https://the-odds-api.com/sports-odds-data/bookmaker-apis.html#eu-bookmakers   

In [2]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# proces variabelen
bestaande_gegevens = pd.read_sql('SELECT * FROM dsa.oddapi_odds WHERE cast(datum as date) >= CURRENT_DATE', sqal_connection)
competities = [['E0','soccer_epl'],['D1','soccer_germany_bundesliga'],['I1','soccer_italy_serie_a']
                ,['SP1','soccer_spain_la_liga'], ['FR1','soccer_france_ligue_one'],['N1','soccer_netherlands_eredivisie']
                ,['B1','soccer_belgium_first_div'] ,['P1','soccer_portugal_primeira_liga']
                ,['T1','soccer_turkey_super_league'],['G1','soccer_greece_super_league']]

# oddapi variabelen
apikey = '01f93cd4e8041d680add34efea99aff5'
regions = 'eu'
markets = 'h2h'
URL = 'https://api.the-odds-api.com'

for competitie in competities:
    tel_nieuwe_odd = 0
    tel_update_odd = 0
    GET = f'/v4/sports/{competitie[1]}/odds/?apiKey={apikey}&regions={regions}&markets={markets}'

    # haal alle odds per bookamer op
    wedstrijden = requests.get(f"{URL}{GET}").json()
    
    for wedstrijd in wedstrijden:
        bookmakers = wedstrijd['bookmakers']
        
        for bookmaker in bookmakers:
            wedkantoor = (bookmaker['key'])
            h_odd_price = bookmaker['markets'][0]['outcomes'][0]['price']
            h_odd = [wedstrijd['commence_time'],wedstrijd['home_team'],wedstrijd['away_team'], wedkantoor, h_odd_price] 
            
            # Update odd indien wedstrijd al voorkomt, anders nieuwe odd toevoegen aan db
            nieuw_wel_niet = True
            for index, bestaand_gegeven in bestaande_gegevens.iterrows():
                if h_odd[0] == bestaand_gegeven[1] and h_odd[1] == bestaand_gegeven[2] and h_odd[3] == bestaand_gegeven[4]:
                    nieuw_wel_niet = False
                    if str(h_odd[4]) != str(bestaand_gegeven[5]):
                        tel_update_odd += 1
                        psy_cursor.execute("""
                                            UPDATE dsa.oddapi_odds 
                                            SET h_odd_price = %s
                                            WHERE datum = %s and hometeam = %s and wedkantoor = %s;"""
                                            ,(h_odd[4], h_odd[0], h_odd[1], h_odd[3]))
            if nieuw_wel_niet == True: 
                tel_nieuwe_odd += 1
                psy_cursor.execute("""
                                    INSERT INTO dsa.oddapi_odds (div, datum, hometeam ,awayteam, wedkantoor, h_odd_price)
                                    VALUES (%s, %s, %s, %s, %s, %s);"""
                                    ,(competitie[0], h_odd[0], h_odd[1], h_odd[2], h_odd[3], h_odd[4]))

    print(f'aantal nieuwe odd in {competitie[0]}: {tel_nieuwe_odd}')
    print(f'aantal updates in {competitie[0]}: {tel_update_odd}')

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close()

{'message': 'Request quota has been reached. See usage plans at https://the-odds-api.com'}


TypeError: string indices must be integers, not 'str'

In [1]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2

# proces variabelen
competities = [['E0','soccer_epl'],['D1','soccer_germany_bundesliga'],['I1','soccer_italy_serie_a']
                ,['SP1','soccer_spain_la_liga'], ['FR1','soccer_france_ligue_one'],['N1','soccer_netherlands_eredivisie']
                ,['B1','soccer_belgium_first_div'] ,['P1','soccer_portugal_primeira_liga']
                ,['T1','soccer_turkey_super_league'],['G1','soccer_greece_super_league']]

# oddapi variabelen
apikey = '01f93cd4e8041d680add34efea99aff5'
regions = 'eu'
markets = 'h2h'
URL = 'https://api.the-odds-api.com'


GET = f'/v4/sports/soccer_germany_bundesliga/odds/?apiKey={apikey}&regions={regions}&markets={markets}'

# haal alle odds per bookamer op
wedstrijden = requests.get(f"{URL}{GET}").json()
for wedstrijd in wedstrijden:
    bookmakers = wedstrijd['bookmakers']
        
    for bookmaker in bookmakers:
        wedkantoor = (bookmaker['key'])
        if wedstrijd['home_team'] == bookmaker['markets'][0]['outcomes'][0]['name']:
            h_odd_price = bookmaker['markets'][0]['outcomes'][0]['price']
        else: 
            h_odd_price = bookmaker['markets'][0]['outcomes'][1]['price']
        h_odd = [wedstrijd['commence_time'],wedstrijd['home_team'],wedstrijd['away_team'], wedkantoor, h_odd_price] 

        print(h_odd)

['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'marathonbet', 10.5]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'livescorebet_eu', 9.5]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'mybookieag', 9.5]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'betclic', 9.75]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'onexbet', 11.1]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'suprabets', 10.0]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'sport888', 11.0]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'pinnacle', 9.79]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'williamhill', 10.0]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'nordicbet', 10.5]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'betsson', 11.0]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'betfair_ex_eu', 12.0]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'unibet_eu', 10.5]
['2023-11-24T19:30:00Z', 'FC Koln', 'Bayern Munich', 'every